# Introduction

In [1]:
from ipywebcam import WebCamWidget

In [2]:
w = WebCamWidget(
    iceServers=[{"urls": "turn:190.92.221.222", "username": "admin", "credential": "123456"}],
    constraints={
        "video": {
            "frameRate": { "max": 10 },
            "width": 256,
            "height": 192,
        },
    },
)
w

WebCamWidget(constraints={'video': {'frameRate': {'max': 10}, 'width': 256, 'height': 192}}, iceServers=[{'url…

In [3]:
w.video_input_device_id

'798221879d265b08da49754fb3f1db1670548e633067d0633ca7a71fcc980560'

In [4]:
w.video_input_selector

Dropdown(description='Video Input', options=(), value=None)

In [5]:
w.audio_input_selector

Dropdown(description='Audio Input', options=(), value=None)

In [6]:
w.audio_output_selector

Dropdown(description='Audio Output', options=(), value=None)

In [7]:
w.audio_output_device_id

In [8]:
from ipywebcam import WebCamRecorder
recorder = WebCamRecorder(w, 'camera.mp4')
recorder.start()

In [9]:
import cv2
from av import VideoFrame
async def transform(frame, ctx):
    img = frame.to_ndarray(format="bgr24")
    # prepare color
    img_color = cv2.pyrDown(cv2.pyrDown(img))
    for _ in range(6):
        img_color = cv2.bilateralFilter(img_color, 9, 9, 7)
    img_color = cv2.pyrUp(cv2.pyrUp(img_color))

    # prepare edges
    img_edges = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img_edges = cv2.adaptiveThreshold(
        cv2.medianBlur(img_edges, 7),
        255,
        cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY,
        9,
        2,
    )
    img_edges = cv2.cvtColor(img_edges, cv2.COLOR_GRAY2RGB)

    # combine color and edges
    img = cv2.bitwise_and(img_color, img_edges)

    # rebuild a VideoFrame, preserving timing information
    new_frame = VideoFrame.from_ndarray(img, format="bgr24")
    # new_frame.pts = frame.pts
    # new_frame.time_base = frame.time_base
    return new_frame

transformer = w.add_video_transformer(transform)

In [10]:
w.remove_video_transformer(transformer)

In [11]:
recorder.stop()

In [15]:
w.close_peers()